In [18]:
import torch
import torch.nn as nn
import numpy as np
import time
import math
from matplotlib import pyplot
import pandas as pd

torch.manual_seed(0)
np.random.seed(0)

# S is the source sequence length
# T is the target sequence length
# N is the batch size
# E is the feature number

#src = torch.rand((10, 32, 512)) # (S,N,E)
#tgt = torch.rand((20, 32, 512)) # (T,N,E)
#out = transformer_model(src, tgt)

input_window = 120  # number of input steps
output_window = 1  # number of prediction steps, in this model its fixed to one
batch_size = 512
device = torch.device("cuda")


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5500):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() *
            (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        #pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]


class TransAm(nn.Module):
    def __init__(self, feature_size=250, num_layers=1, dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'

        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size,
                                                        nhead=10,
                                                        dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer,
                                                         num_layers=num_layers)
        self.decoder = nn.Linear(feature_size, 1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,
                                          self.src_mask)  #, self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(
            mask == 1, float(0.0))
        return mask


# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L - tw):
        train_seq = input_data[i:i + tw]
        train_label = input_data[i + output_window:i + tw + output_window]
        inout_seq.append((train_seq, train_label))
    return torch.FloatTensor(inout_seq)


def get_data(name):
    # construct a littel toy dataset
    time = np.arange(0, 400, 0.1)
    data = pd.read_csv('../data/399300.csv')
    amplitude = data[name].values
    from sklearn.preprocessing import MinMaxScaler

    #loading weather data from a file
    #from pandas import read_csv
    #series = read_csv('daily-min-temperatures.csv', header=0, index_col=0, parse_dates=True, squeeze=True)

    # looks like normalizing input values curtial for the model
    scaler = MinMaxScaler(feature_range=(-1, 1))
    #amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
    amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)

    sampels = int(len(amplitude)*0.8)
    train_data = amplitude[:sampels]
    test_data = amplitude[sampels:]

    # convert our train data into a pytorch train tensor
    #train_tensor = torch.FloatTensor(train_data).view(-1)
    # todo: add comment..
    train_sequence = create_inout_sequences(train_data, input_window)
    train_sequence = train_sequence[:
                                    -output_window]  #todo: fix hack? -> din't think this through, looks like the last n sequences are to short, so I just remove them. Hackety Hack..

    #test_data = torch.FloatTensor(test_data).view(-1)
    test_data = create_inout_sequences(test_data, input_window)
    test_data = test_data[:-output_window]  #todo: fix hack?

    return train_sequence.to(device), test_data.to(device),scaler


def get_batch(source, i, batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i + seq_len]
    input = torch.stack(
        torch.stack([item[0] for item in data]).chunk(input_window,1))  # 1 is feature size
    target = torch.stack(
        torch.stack([item[1] for item in data]).chunk(input_window, 1))
    return input, target


def train(train_data,epoch):
    model.train()  # Turn on the train mode \o/
    total_loss = 0.
    start_time = time.time()
    

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        data, targets = get_batch(train_data, i, batch_size)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.7)
        optimizer.step()

        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                      epoch, batch,
                      len(train_data) // batch_size,
                      scheduler.get_lr()[0], elapsed * 1000 / log_interval,
                      cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()


def plot_and_loss(eval_model, data_source, epoch):
    eval_model.eval()
    total_loss = 0.
    test_result = torch.Tensor(0)
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i, 1)
            output = eval_model(data)
            total_loss += criterion(output, target).item()
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()),
                                    0)
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)

    #test_result = test_result.cpu().numpy() -> no need to detach stuff..
    len(test_result)

    pyplot.plot(test_result, color="red", label="prediction")
    pyplot.plot(truth[:len(test_result)], color="blue", label="truth")
    pyplot.plot(test_result - truth, color="green", label="error")
    pyplot.legend()
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-epoch%d.png' % epoch)
    pyplot.close()

    return total_loss / i


# predict the next n steps based on the input data
def predict_future(eval_model, data_source, steps):
    eval_model.eval()
    total_loss = 0.
    test_result = torch.Tensor(0)
    truth = torch.Tensor(0)
    data, _ = get_batch(data_source, 0, 1)
    with torch.no_grad():
        for i in range(0, steps):
            output = eval_model(data[-input_window:])
            data = torch.cat((data, output[-1:]))

    data = data.cpu().view(-1)

    # I used this plot to visualize if the model pics up any long therm structure within the data.
    pyplot.plot(data, color="red")
    pyplot.plot(data[:input_window], color="blue")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-future%d.png' % steps)
    pyplot.close()


def evaluate(eval_model, data_source):
    eval_model.eval()  # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i, eval_batch_size)
            output = eval_model(data)
            total_loss += len(data[0]) * criterion(output,
                                                   targets).cpu().item()
    return total_loss / len(data_source)


In [19]:
def train_and_get(name):
    input_window = 120  # number of input steps
    output_window = 1  # number of prediction steps, in this model its fixed to one
    batch_size = 512
    device = torch.device("cuda")
    train_data, val_data, scaler = get_data(name)
    global model 
    model = TransAm().to(device)

    global criterion
    criterion = nn.MSELoss()
    lr = 0.0004  # learning rate
    #optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    global optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    global scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

    best_val_loss = float("inf")
    global epochs
    epochs = 200  # The number of epochs
    best_model = None

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train(train_data,epoch)

        if (epoch % 100 == 0):
            val_loss = plot_and_loss(model, val_data, epoch)
            predict_future(model, val_data, 5)
        else:
            val_loss = evaluate(model, val_data)

        if(epoch % 50 ==0):
            print('-' * 89)
            print(
                '| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'
                .format(epoch, (time.time() - epoch_start_time), val_loss,
                        math.exp(val_loss)))
            print('-' * 89)
        scheduler.step()
    
    def get_predict(eval_model, data_source):
        eval_model.eval()
        total_loss = 0.
        test_result = torch.Tensor(0)
        truth = torch.Tensor(0)
        with torch.no_grad():
            for i in range(0, len(data_source) - 1):
                data, target = get_batch(data_source, i, 1)
                output = eval_model(data)
                total_loss += criterion(output, target).item()
                test_result = torch.cat((test_result, output[-1].view(-1).cpu()),
                                        0)
                truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)

        #test_result = test_result.cpu().numpy() -> no need to detach stuff..
        return scaler.inverse_transform(test_result.view(-1, 1)), scaler.inverse_transform(truth.view(-1, 1))

    
    return get_predict(model, val_data)


In [22]:
for i in ['open','close','high','low']:
    predict,truth = train_and_get(i)

d:\tools\anaconda3\envs\data\lib\site-packages\torch\optim\lr_scheduler.py:381: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |     7/    7 batches | lr 0.000400 | 94.13 ms | loss 1.80968 | ppl     6.11
| epoch   2 |     7/    7 batches | lr 0.000384 | 55.26 ms | loss 0.11886 | ppl     1.13
| epoch   3 |     7/    7 batches | lr 0.000376 | 55.55 ms | loss 0.05973 | ppl     1.06
| epoch   4 |     7/    7 batches | lr 0.000369 | 55.83 ms | loss 0.05601 | ppl     1.06
| epoch   5 |     7/    7 batches | lr 0.000362 | 55.36 ms | loss 0.04783 | ppl     1.05
| epoch   6 |     7/    7 batches | lr 0.000354 | 55.64 ms | loss 0.09573 | ppl     1.10
| epoch   7 |     7/    7 batches | lr 0.000347 | 55.38 ms | loss 0.13908 | ppl     1.15
| epoch   8 |     7/    7 batches | lr 0.000340 | 55.51 ms | loss 0.11642 | ppl     1.12
| epoch   9 |     7/    7 batches | lr 0.000333 | 54.99 ms | loss 0.08932 | ppl     1.09
| epoch  10 |     7/    7 batches | lr 0.000327 | 55.48 ms | loss 0.05456 | ppl     1.06
| epoch  11 |     7/    7 batches | lr 0.000320 | 55.38 ms | loss 0.05780 | ppl     1.06
| epoch  12 |     7/ 

d:\tools\anaconda3\envs\data\lib\site-packages\torch\optim\lr_scheduler.py:381: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |     7/    7 batches | lr 0.000400 | 56.20 ms | loss 4.70828 | ppl   110.86
| epoch   2 |     7/    7 batches | lr 0.000384 | 56.40 ms | loss 0.85826 | ppl     2.36
| epoch   3 |     7/    7 batches | lr 0.000376 | 56.14 ms | loss 0.77974 | ppl     2.18
| epoch   4 |     7/    7 batches | lr 0.000369 | 55.96 ms | loss 0.29605 | ppl     1.34
| epoch   5 |     7/    7 batches | lr 0.000362 | 56.38 ms | loss 0.24820 | ppl     1.28
| epoch   6 |     7/    7 batches | lr 0.000354 | 56.45 ms | loss 0.18290 | ppl     1.20
| epoch   7 |     7/    7 batches | lr 0.000347 | 55.97 ms | loss 0.08408 | ppl     1.09
| epoch   8 |     7/    7 batches | lr 0.000340 | 56.18 ms | loss 0.14601 | ppl     1.16
| epoch   9 |     7/    7 batches | lr 0.000333 | 68.00 ms | loss 0.09350 | ppl     1.10
| epoch  10 |     7/    7 batches | lr 0.000327 | 56.23 ms | loss 0.12014 | ppl     1.13
| epoch  11 |     7/    7 batches | lr 0.000320 | 55.98 ms | loss 0.20496 | ppl     1.23
| epoch  12 |     7/ 

d:\tools\anaconda3\envs\data\lib\site-packages\torch\optim\lr_scheduler.py:381: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |     7/    7 batches | lr 0.000400 | 54.96 ms | loss 3.95233 | ppl    52.06
| epoch   2 |     7/    7 batches | lr 0.000384 | 56.19 ms | loss 0.17473 | ppl     1.19
| epoch   3 |     7/    7 batches | lr 0.000376 | 57.15 ms | loss 0.11001 | ppl     1.12
| epoch   4 |     7/    7 batches | lr 0.000369 | 56.67 ms | loss 0.06738 | ppl     1.07
| epoch   5 |     7/    7 batches | lr 0.000362 | 56.43 ms | loss 0.05926 | ppl     1.06
| epoch   6 |     7/    7 batches | lr 0.000354 | 56.34 ms | loss 0.04810 | ppl     1.05
| epoch   7 |     7/    7 batches | lr 0.000347 | 56.53 ms | loss 0.13508 | ppl     1.14
| epoch   8 |     7/    7 batches | lr 0.000340 | 56.43 ms | loss 0.11431 | ppl     1.12
| epoch   9 |     7/    7 batches | lr 0.000333 | 56.92 ms | loss 0.05483 | ppl     1.06
| epoch  10 |     7/    7 batches | lr 0.000327 | 56.74 ms | loss 0.03017 | ppl     1.03
| epoch  11 |     7/    7 batches | lr 0.000320 | 56.30 ms | loss 0.02295 | ppl     1.02
| epoch  12 |     7/ 

d:\tools\anaconda3\envs\data\lib\site-packages\torch\optim\lr_scheduler.py:381: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |     7/    7 batches | lr 0.000400 | 56.34 ms | loss 4.15065 | ppl    63.48
| epoch   2 |     7/    7 batches | lr 0.000384 | 56.67 ms | loss 0.35673 | ppl     1.43
| epoch   3 |     7/    7 batches | lr 0.000376 | 56.42 ms | loss 0.30213 | ppl     1.35
| epoch   4 |     7/    7 batches | lr 0.000369 | 56.16 ms | loss 0.16104 | ppl     1.17
| epoch   5 |     7/    7 batches | lr 0.000362 | 56.27 ms | loss 0.29851 | ppl     1.35
| epoch   6 |     7/    7 batches | lr 0.000354 | 56.23 ms | loss 0.16758 | ppl     1.18
| epoch   7 |     7/    7 batches | lr 0.000347 | 56.39 ms | loss 0.20301 | ppl     1.23
| epoch   8 |     7/    7 batches | lr 0.000340 | 56.36 ms | loss 0.13585 | ppl     1.15
| epoch   9 |     7/    7 batches | lr 0.000333 | 56.03 ms | loss 0.12303 | ppl     1.13
| epoch  10 |     7/    7 batches | lr 0.000327 | 56.64 ms | loss 0.14251 | ppl     1.15
| epoch  11 |     7/    7 batches | lr 0.000320 | 56.20 ms | loss 0.03343 | ppl     1.03
| epoch  12 |     7/ 

In [23]:
result.to_csv('result.csv')